# mastodon API

read data of accounts you are following and store it in excel file.

+ utilizes `Mastodon.py` package to read account information
+ `pandas` is used to create excel file


background links, references

+ [Mastodon.py](https://pypi.org/project/Mastodon.py/)
  + [pypi](https://pypi.org/project/Mastodon.py/)
  + [doc](https://mastodonpy.readthedocs.io/en/stable/)
  + [repo](https://github.com/halcy/Mastodon.py)
+ [mastodon](https://joinmastodon.org/)
  + [API getting started](https://docs.joinmastodon.org/client/intro/)
  + [pagination](https://docs.joinmastodon.org/api/guidelines/#pagination)

**setup**

```shell
pip install -r requirements.txt
```

In [ ]:
sSecretFN='.env.py.secret'

create an access token through profile, go to `</> Development`

+ `New Application`
  + provide meaningful application name
  + define scopes, grant permission for application
  + `submit`
+ click on application name
  + copy value of `your access token`
  + store it to file `sSecretFN`
  + `save changes`

minimal content of `sSecretFN`

```
access_token="<your access token>"
api_base_url="https://<instance name>"
```


create an access token through python, do this only once

```python
from mastodon import Mastodon

oMstdn = Mastodon(
    client_id = '<your app>',
    api_base_url = 'https://<your instance>'
)
oMstdn.log_in(
    'youremail',
    'superseretpasswork',
    to_file = '.env.py.secret'
)
```

**THE MIT license (MIT)**

Copyright © 2022 Walter Obweger (WRO)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
import pandas
import time

In [ ]:
import pprint
import traceback
import random

In [ ]:
dfFollows=pandas.read_csv('~/Downloads/following_accounts.csv')

In [ ]:
dfFollows.head()

In [ ]:
dfAcc=dfFollows.set_index(keys=['Account address'])

In [ ]:
srAcc=dfFollows['Account address']

In [ ]:
lAcc=list(srAcc)

In [ ]:
dfAcc.head()

In [ ]:
dfHist=pandas.read_excel('~/Downloads/Histodons.xlsx',
                header=None,
                names=['zTime','name','account','interest','area','research','description','list','twitter','mastodonURL'],
                skiprows=11)

In [ ]:
dfHist.head()

In [ ]:
lAccToAdd=[]
for iIdx,oItem in dfHist.iterrows():
    sAccount=oItem['account']
    if sAccount.startswith('@'):
        sAccount=sAccount[1:]
    if sAccount in lAcc:
        continue
    else:
        dNewAcc={
            'Account address':sAccount,
            'Show boosts':True,
            'Notify on new posts':False,
            'Languages':None,
        }
        lAccToAdd.append(dNewAcc)

In [ ]:
pprint.pp(lAccToAdd)

In [ ]:
dfNewAcc=pandas.DataFrame.from_dict(lAccToAdd)

In [ ]:
dfNewAcc=pandas.DataFrame(lAccToAdd,columns=['Account address','Show boosts','Notify on new posts','Languages'])

In [ ]:
dfNewAcc.head()

In [ ]:
dfFollowsNew=pandas.concat([dfFollows,dfNewAcc],axis=0,ignore_index=True)

In [ ]:
print(dfFollowsNew.shape)

In [ ]:
dfFollowsNew.to_csv('~/Downloads/following_accounts_new.csv')

In [ ]:
from mastodon import Mastodon
from dotenv import load_dotenv
import os

In [ ]:
iVerbose=20

In [ ]:
load_dotenv(".env.pyB.secret")
print(os.environ.get('api_base_url'))

In [ ]:
oMstdn = Mastodon(
    access_token = os.environ.get('access_token'),
    api_base_url = os.environ.get('api_base_url')
)

In [ ]:
sVer=oMstdn.retrieve_mastodon_version()
print(sVer)

In [ ]:
dMe=oMstdn.me()

In [ ]:
pprint.pp(dMe)

In [ ]:
print(dMe.id)

In [ ]:
dMe.created_at.strftime('%Y-%m-%d')

In [ ]:
print(dMe.url)

In [ ]:
lFollowing=oMstdn.account_following(dMe.id)

In [ ]:
print(list(lFollowing[0].keys()))

In [ ]:
def addAccountToDict(lFollowing,dFollowing,lFollow):
    for dAcc in lFollowing:
        dFollowing[dAcc.id]=dAcc.url
        dFollowing[dAcc.url]=dAcc.id
        dA={
            'id':dAcc.id,
            'username':dAcc.username,
            'display_name':dAcc.display_name,
            'acct':dAcc.acct,
            'created_at':dAcc.created_at.strftime('%Y-%m-%d'),
            'statuses_count':dAcc.statuses_count,
            'followers_count':dAcc.followers_count,
            'following_count':dAcc.following_count,
            'discoverable':dAcc.discoverable,
            'locked':dAcc.locked,
            'group':dAcc.group,
            'bot':dAcc.bot,
            'note':dAcc.note,
            'url':dAcc.url,
            'header':dAcc.header,

        }
        if dAcc.last_status_at is not None:
            dAcc['last_status_at']=dAcc.last_status_at.strftime('%Y-%m-%d')
        lFollow.append(dA)

In [ ]:
dFollowing={}
lFollow=[]
try:
    while (lFollowing is not None):
        addAccountToDict(lFollowing,dFollowing,lFollow)
        lFollowingNext=oMstdn.fetch_next(lFollowing)
        if iVerbose>20:
            print(len(lFollowingNext))
        lFollowing=lFollowingNext
        time.sleep(0.1+random.random()*0.25)
except:
    print(traceback.format_exc())

if iVerbose>0:
    print('following count:%d'%(len(dFollowing)))
if iVerbose>20:
    pprint.pp(dFollowing)

In [ ]:
print('following count:%d'%(len(lFollow)))

In [ ]:
dfFollowing=pandas.DataFrame(data=lFollow,index=None,dtype='string')

In [ ]:
with pandas.ExcelWriter('mstdnF.out.xlsx',mode='w') as oXlsWr:
    dfFollowing.to_excel(oXlsWr,sheet_name='following',index=False)

In [ ]:
def doSearchAccount(lAcc,dFollowing={},lAccToFollow=[],iVerbose=0):
    for dAcc in lAcc:
        iId=dAcc.id
        if iId in dFollowing:
            continue 
        else:
            sUrl=dAcc.url
            if sUrl in dFollowing:
                continue 
            else:
                if iVerbose>10:
                    print(sUrl,"not in dFollowing")
                lAccToFollow.append(dAcc.acct)
                if iVerbose>0:
                    print('new account to follow %s'%(dAcc.acct))
    return lAccToFollow

In [ ]:
lAccToFollow=[]
for iIdx,oItem in dfHist.iterrows():
    sAccount=oItem['account']
    if iVerbose>10:
        print(sAccount)
    dSearch=oMstdn.search(sAccount)
    if 'accounts' in dSearch:
        doSearchAccount(dSearch['accounts'],
                    dFollowing=dFollowing,
                    lAccToFollow=lAccToFollow,
                    iVerbose=iVerbose)

In [ ]:
list(dFollowing.keys())

In [ ]:
pprint.pp(lAccToFollow)

In [ ]:
sAccount=dfHist['account'].iloc[1]
print(sAccount)

In [ ]:
dRet=oMstdn.search(sAccount)

In [ ]:
pprint.pprint(dRet)

In [ ]:
iId=dRet['accounts'][0]['id']
print(iId)

In [ ]:
print(dRet['accounts'][0]['acct'])

In [ ]:
dRet2=oMstdn.account_follow(dRet['accounts'][0]['id'])
#dRet=oMstdn.account_follow(dRet['accounts'][0]['url'])
pprint.pprint(dRet2)

In [ ]:
dRet3=oMstdn.account_follow(dRet['accounts'][0]['id'],reblogs=True)


In [ ]:
dRet4=oMstdn.follows(sAccount[1:])
#dRet=oMstdn.follows(dRet['accounts'][1]['url'])
print(dRet4)

In [ ]:
dRet2=oMstdn.status_reply(to_status=dRet,status='reply to toot',in_reply_to_id=dRet['id'])

In [ ]:
dRet3=oMstdn.status_favourite(dRet2)

In [ ]:
dRet=oMstdn.trends(limit=5)

In [ ]:
print(dRet)

In [ ]:
dRet=oMstdn.search_v2("tags/digitalhumanities")

In [ ]:
print(dRet)